In [8]:
import random
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [9]:

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [10]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

In [11]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [12]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [13]:


le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])

/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [15]:
price_df = price_data.copy()

In [16]:
# Class Definitions

class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)

        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(torch.nn.Module):
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        residual = x - moving_mean
        return moving_mean, residual

class LTSF_DLinear(torch.nn.Module):
    def __init__(self, window_size, forecast_size, kernel_size, individual, feature_size):
        super(LTSF_DLinear, self).__init__()
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.decomposition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList()
            self.Linear_Trend = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forecast_size))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1 / self.window_size) * torch.ones([self.forecast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forecast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1 / self.window_size) * torch.ones([self.forecast_size, self.window_size]))
        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forecast_size)
            self.Linear_Trend.weight = torch.nn.Parameter((1 / self.window_size) * torch.ones([self.forecast_size, self.window_size]))
            self.Linear_Seasonal = torch.nn.Linear(self.window_size, self.forecast_size)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1 / self.window_size) * torch.ones([self.forecast_size, self.window_size]))

    def forward(self, x):
        trend_init, seasonal_init = self.decomposition(x)
        trend_init, seasonal_init = trend_init.permute(0, 2, 1), seasonal_init.permute(0, 2, 1)
        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forecast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forecast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])
        else:
            trend_output = self.Linear_Trend(trend_init)
            seasonal_output = self.Linear_Seasonal(seasonal_init)
        x = seasonal_output + trend_output
        ################
        #x = torch.relu(x) # 음수 값 제거
        ################
        return x.permute(0, 2, 1)

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [17]:
price_df.head(3)

,corp,loca,product,product_loca,item,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,...,2023-02-22,2023-02-23,2023-02-24,2023-02-25,2023-02-26,2023-02-27,2023-02-28,2023-03-01,2023-03-02,2023-03-03
0,A,J,TG,8,TG A J,0.0,0.0,1728.0,1408.0,1250.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,A,S,TG,9,TG A S,0.0,0.0,2526.0,2134.0,2075.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,B,J,TG,8,TG B J,0.0,0.0,1692.0,1516.0,1471.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0


In [18]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[5:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(price_df)

In [21]:
# Define the window size, forecast size, and batch size
window_size = 72   
forecast_size = 28
batch_size = 1024
epoch_count = 777
lr = 0.001
min_delta = 0.001
patience = 10

# Model Train 

In [22]:

future_predictions_by_id = {}
loss_history = {}
# Iterate through the data by ID
for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
    # Standardizing the data
    mean_ = np.mean(data)
    std_ = np.std(data)
    
    # Check if std_ is zero and handle standardization accordingly
    if std_ == 0:
        standardized_data = data
    else:
        standardized_data = (data - mean_) / std_
        
    individual_loss_history = []
    
    # Create DataLoader
    train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
    best_loss = float('inf')
    no_improvement_count = 0

    # Training the model
    DLinear_model = LTSF_DLinear(window_size=window_size, forecast_size=forecast_size, kernel_size=21, individual=False, feature_size=1)
    DLinear_model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
    
    for epoch in range(1, epoch_count + 1):
        loss_list = []
        DLinear_model.train()
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
            optimizer.zero_grad()
            output = DLinear_model(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        
        if((epoch % 10) == 0):
            avg_loss = np.mean(loss_list)
            individual_loss_history.append(avg_loss)
            if avg_loss + min_delta < best_loss:
                best_loss = avg_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    break

    loss_history[id_val] = individual_loss_history

    # Predicting the future 15 days using the last window of data
    last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
    future_prediction = DLinear_model(last_window_data)

    # Converting the prediction back to the original scale
    if std_ == 0:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy()
    else:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

    # Store the prediction
    future_predictions_by_id[id_val] = future_prediction
future_predictions_by_id

  0%|          | 0/39 [00:00<?, ?it/s]

{'TG A J': array([2749.677  , 1235.0486 , 2547.6096 , 2915.0786 , 2700.4097 ,
        3143.7869 , 2981.5142 , 2703.125  ,  988.87463, 2450.4812 ,
        2911.93   , 2554.9956 , 3140.414  , 3117.8936 , 2804.2688 ,
        1328.7708 , 2662.5132 , 3056.026  , 2979.4956 , 3337.4626 ,
        2921.7588 , 2766.4163 , 1264.8949 , 2639.893  , 3034.0784 ,
        2789.5945 , 3225.6746 , 3241.2495 ], dtype=float32),
 'TG A S': array([4014.8213,  851.6885, 3656.2693, 3812.5522, 4066.9536, 3960.2322,
        4124.9243, 4271.727 , 1108.7908, 4017.113 , 4125.4287, 4221.1226,
        3934.4336, 4112.9243, 4536.8223, 1261.3645, 3846.4004, 4147.279 ,
        4491.1606, 4174.438 , 4296.7456, 4268.428 , 1362.7766, 3981.3806,
        4308.279 , 4245.6914, 4287.527 , 4577.167 ], dtype=float32),
 'TG B J': array([3044.9106 , 1458.6118 , 3105.253  , 2326.035  , 3670.9827 ,
        3806.1262 , 3383.3154 , 3071.355  ,  966.84937, 2893.0522 ,
        3581.1116 , 3893.3262 , 4427.5786 , 3981.4265 , 3186.3186 ,


In [28]:
(future_predictions_by_id.values)

<function dict.values>

In [32]:
submit = pd.read_csv('./../../DATA/sample_submission.csv')
submit

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
5,TG_A_J_20230309,0
6,TG_A_J_20230310,0
7,TG_A_J_20230311,0
8,TG_A_J_20230312,0
9,TG_A_J_20230313,0


In [41]:
findata = []
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    findata.extend(rounded_predictions)

submit['answer'] = findata
submit.loc[submit['answer'] <= 0, 'answer'] = 0
    

In [44]:
submit.loc[submit.index % 7 == 1, 'answer'] = 0

In [46]:
submit.to_csv('../../DATA/SUBMIT/Dlinear1.csv', index=False)